# Phase 2: Exploratory Data Analysis (EDA)

This notebook performs comprehensive exploratory data analysis on the hotel booking dataset to:
1. Understand data structure and quality
2. Identify patterns and relationships
3. Discover factors influencing cancellations
4. Prepare insights for feature engineering

## Objectives
- Univariate analysis of all features
- Bivariate analysis with target variable
- Correlation analysis
- Missing value analysis
- Key insights extraction


In [ ]:
# Install required packages
%pip install pandas numpy matplotlib seaborn plotly -q


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")


## Step 1: Load Data

Load data from CSV file or MongoDB (from previous notebook).


In [ ]:
# Option 1: Load from CSV (if you have the file)
csv_path = "/content/hotel_bookings.csv"  # Colab path
# csv_path = "../data/hotel_bookings.csv"  # Local path

# Option 2: Load from MongoDB (from previous notebook)
# from pymongo import MongoClient
# MONGODB_URI = "your_connection_string"
# client = MongoClient(MONGODB_URI)
# db = client["hotel_bookings"]
# collection = db["bookings"]
# cursor = collection.find()
# df = pd.DataFrame(list(cursor))
# if '_id' in df.columns:
#     df = df.drop('_id', axis=1)

# Load data
try:
    df = pd.read_csv(csv_path)
    print(f"✓ Data loaded from CSV: {csv_path}")
except FileNotFoundError:
    print("CSV not found. Please load from MongoDB or update path.")
    df = None

if df is not None:
    print(f"\nDataset shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


## Step 2: Data Overview


In [ ]:
# Basic information
print("=== Dataset Information ===")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumn names and types:")
print(df.dtypes)
print(f"\nFirst few rows:")
display(df.head())


In [ ]:
# Missing values analysis
print("=== Missing Values Analysis ===")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing Count': missing.values,
    'Missing Percentage': missing_pct.values
}).sort_values('Missing Count', ascending=False)

missing_df = missing_df[missing_df['Missing Count'] > 0]
if len(missing_df) > 0:
    display(missing_df)
    print(f"\nTotal columns with missing values: {len(missing_df)}")
else:
    print("✓ No missing values found!")


In [ ]:
# Statistical summary
print("=== Statistical Summary (Numerical Features) ===")
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
display(df[numerical_cols].describe())


In [ ]:
# Categorical features summary
print("=== Categorical Features Summary ===")
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols[:5]:  # Show first 5
    print(f"\n{col}:")
    print(df[col].value_counts().head(10))


In [ ]:
# Target distribution
target_col = 'is_canceled'
cancellation_counts = df[target_col].value_counts()
cancellation_pct = df[target_col].value_counts(normalize=True) * 100

print("=== Target Variable Distribution ===")
print(f"Not Cancelled (0): {cancellation_counts[0]:,} ({cancellation_pct[0]:.2f}%)")
print(f"Cancelled (1): {cancellation_counts[1]:,} ({cancellation_pct[1]:.2f}%)")
print(f"Total: {len(df):,}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
axes[0].bar(['Not Cancelled', 'Cancelled'], cancellation_counts.values, 
            color=['#2ecc71', '#e74c3c'], alpha=0.7)
axes[0].set_title('Cancellation Count', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
for i, v in enumerate(cancellation_counts.values):
    axes[0].text(i, v, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Pie chart
axes[1].pie(cancellation_counts.values, labels=['Not Cancelled', 'Cancelled'],
           autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'], startangle=90)
axes[1].set_title('Cancellation Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


## Step 4: Univariate Analysis - Numerical Features


In [ ]:
# Key numerical features to analyze
key_numerical = ['lead_time', 'adr', 'stays_in_weekend_nights', 
                 'stays_in_week_nights', 'adults', 'children', 'babies',
                 'previous_cancellations', 'previous_bookings_not_canceled']

# Filter to existing columns
key_numerical = [col for col in key_numerical if col in df.columns]

# Distribution plots
n_cols = 3
n_rows = (len(key_numerical) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
axes = axes.flatten()

for i, col in enumerate(key_numerical):
    axes[i].hist(df[col].dropna(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[i].set_title(f'{col} Distribution', fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

# Hide extra subplots
for i in range(len(key_numerical), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Box plots for key numerical features
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

features_to_plot = ['lead_time', 'adr', 'stays_in_weekend_nights', 'stays_in_week_nights']
for i, col in enumerate(features_to_plot):
    if col in df.columns:
        row = i // 2
        col_idx = i % 2
        axes[row, col_idx].boxplot(df[col].dropna(), vert=True)
        axes[row, col_idx].set_title(f'{col} Box Plot', fontweight='bold')
        axes[row, col_idx].set_ylabel(col)
        axes[row, col_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Step 5: Univariate Analysis - Categorical Features


In [ ]:
# Key categorical features
key_categorical = ['hotel', 'meal', 'market_segment', 'distribution_channel',
                   'deposit_type', 'customer_type', 'reservation_status']

key_categorical = [col for col in key_categorical if col in df.columns]

# Bar plots for categorical features
n_cols = 2
n_rows = (len(key_categorical) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5*n_rows))
axes = axes.flatten()

for i, col in enumerate(key_categorical):
    value_counts = df[col].value_counts().head(10)
    axes[i].barh(range(len(value_counts)), value_counts.values, color='coral', alpha=0.7)
    axes[i].set_yticks(range(len(value_counts)))
    axes[i].set_yticklabels(value_counts.index)
    axes[i].set_title(f'{col} Distribution', fontweight='bold')
    axes[i].set_xlabel('Count')
    axes[i].invert_yaxis()

# Hide extra subplots
for i in range(len(key_categorical), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()


## Step 6: Bivariate Analysis - Cancellation by Features


In [ ]:
# Cancellation rate by hotel type
print("=== Cancellation Rate by Hotel Type ===")
hotel_cancel = df.groupby('hotel')['is_canceled'].agg(['count', 'sum', 'mean']).reset_index()
hotel_cancel.columns = ['hotel', 'total_bookings', 'cancelled', 'cancellation_rate']
hotel_cancel['cancellation_rate'] = hotel_cancel['cancellation_rate'] * 100
display(hotel_cancel)

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(hotel_cancel['hotel'], hotel_cancel['cancellation_rate'], 
              color=['#3498db', '#e74c3c'], alpha=0.7)
ax.set_title('Cancellation Rate by Hotel Type', fontsize=14, fontweight='bold')
ax.set_ylabel('Cancellation Rate (%)')
ax.set_xlabel('Hotel Type')
for i, (bar, rate) in enumerate(zip(bars, hotel_cancel['cancellation_rate'])):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
           f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Cancellation rate by deposit type
print("=== Cancellation Rate by Deposit Type ===")
deposit_cancel = df.groupby('deposit_type')['is_canceled'].agg(['count', 'sum', 'mean']).reset_index()
deposit_cancel.columns = ['deposit_type', 'total_bookings', 'cancelled', 'cancellation_rate']
deposit_cancel['cancellation_rate'] = deposit_cancel['cancellation_rate'] * 100
deposit_cancel = deposit_cancel.sort_values('cancellation_rate', ascending=False)
display(deposit_cancel)

# Visualization
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(deposit_cancel['deposit_type'], deposit_cancel['cancellation_rate'],
              color='coral', alpha=0.7)
ax.set_title('Cancellation Rate by Deposit Type', fontsize=14, fontweight='bold')
ax.set_xlabel('Cancellation Rate (%)')
for i, (bar, rate) in enumerate(zip(bars, deposit_cancel['cancellation_rate'])):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
           f'{rate:.1f}%', ha='left', va='center', fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Cancellation rate by customer type
print("=== Cancellation Rate by Customer Type ===")
customer_cancel = df.groupby('customer_type')['is_canceled'].agg(['count', 'sum', 'mean']).reset_index()
customer_cancel.columns = ['customer_type', 'total_bookings', 'cancelled', 'cancellation_rate']
customer_cancel['cancellation_rate'] = customer_cancel['cancellation_rate'] * 100
customer_cancel = customer_cancel.sort_values('cancellation_rate', ascending=False)
display(customer_cancel)

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(customer_cancel['customer_type'], customer_cancel['cancellation_rate'],
             color='steelblue', alpha=0.7)
ax.set_title('Cancellation Rate by Customer Type', fontsize=14, fontweight='bold')
ax.set_ylabel('Cancellation Rate (%)')
ax.set_xlabel('Customer Type')
plt.xticks(rotation=45)
for bar, rate in zip(bars, customer_cancel['cancellation_rate']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
           f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Lead time vs cancellation
print("=== Lead Time Analysis ===")
df['lead_time_category'] = pd.cut(df['lead_time'], 
                                  bins=[0, 30, 90, 180, 365, np.inf],
                                  labels=['0-30', '31-90', '91-180', '181-365', '365+'])

lead_cancel = df.groupby('lead_time_category')['is_canceled'].agg(['count', 'sum', 'mean']).reset_index()
lead_cancel.columns = ['lead_time_category', 'total_bookings', 'cancelled', 'cancellation_rate']
lead_cancel['cancellation_rate'] = lead_cancel['cancellation_rate'] * 100
display(lead_cancel)

# Visualization
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(lead_cancel['lead_time_category'].astype(str), 
             lead_cancel['cancellation_rate'], color='mediumpurple', alpha=0.7)
ax.set_title('Cancellation Rate by Lead Time Category', fontsize=14, fontweight='bold')
ax.set_ylabel('Cancellation Rate (%)')
ax.set_xlabel('Lead Time (days)')
for bar, rate in zip(bars, lead_cancel['cancellation_rate']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
           f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
plt.tight_layout()
plt.show()


## Step 7: Correlation Analysis


In [ ]:
# Correlation matrix for numerical features
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
if 'is_canceled' in numerical_features:
    corr_matrix = df[numerical_features].corr()
    
    # Focus on correlation with target
    target_corr = corr_matrix['is_canceled'].sort_values(ascending=False)
    print("=== Correlation with Target Variable (is_canceled) ===")
    display(target_corr)
    
    # Heatmap
    plt.figure(figsize=(16, 12))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix - Numerical Features', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()


In [ ]:
# Top features correlated with cancellation
print("=== Top 10 Features Most Correlated with Cancellation ===")
top_corr = target_corr.drop('is_canceled').abs().sort_values(ascending=False).head(10)
display(pd.DataFrame({
    'Feature': top_corr.index,
    'Correlation': [target_corr[feat] for feat in top_corr.index]
}))

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in [target_corr[feat] for feat in top_corr.index]]
bars = ax.barh(range(len(top_corr)), [target_corr[feat] for feat in top_corr.index], color=colors, alpha=0.7)
ax.set_yticks(range(len(top_corr)))
ax.set_yticklabels(top_corr.index)
ax.set_title('Top 10 Features Correlated with Cancellation', fontsize=14, fontweight='bold')
ax.set_xlabel('Correlation Coefficient')
ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()


## Step 8: Time-based Analysis


In [ ]:
# Cancellation rate by month
print("=== Cancellation Rate by Arrival Month ===")
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
              'July', 'August', 'September', 'October', 'November', 'December']
df['arrival_date_month'] = pd.Categorical(df['arrival_date_month'], categories=month_order, ordered=True)

month_cancel = df.groupby('arrival_date_month')['is_canceled'].agg(['count', 'sum', 'mean']).reset_index()
month_cancel.columns = ['month', 'total_bookings', 'cancelled', 'cancellation_rate']
month_cancel['cancellation_rate'] = month_cancel['cancellation_rate'] * 100
display(month_cancel)

# Visualization
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(month_cancel['month'], month_cancel['cancellation_rate'], 
       marker='o', linewidth=2, markersize=8, color='crimson')
ax.set_title('Cancellation Rate by Arrival Month', fontsize=14, fontweight='bold')
ax.set_ylabel('Cancellation Rate (%)')
ax.set_xlabel('Month')
plt.xticks(rotation=45)
ax.grid(True, alpha=0.3)
for i, (month, rate) in enumerate(zip(month_cancel['month'], month_cancel['cancellation_rate'])):
    ax.text(i, rate + 1, f'{rate:.1f}%', ha='center', va='bottom', fontsize=8)
plt.tight_layout()
plt.show()


## Step 9: Key Insights Summary


In [ ]:
print("=" * 60)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("=" * 60)

print("\n1. TARGET DISTRIBUTION:")
print(f"   - Overall cancellation rate: {df['is_canceled'].mean()*100:.2f}%")
print(f"   - Class imbalance: {'Yes' if abs(df['is_canceled'].mean() - 0.5) < 0.1 else 'No'}")

print("\n2. TOP FACTORS INFLUENCING CANCELLATION:")
if 'deposit_type' in df.columns:
    deposit_high = df.groupby('deposit_type')['is_canceled'].mean().idxmax()
    print(f"   - Deposit Type: {deposit_high} has highest cancellation rate")
if 'lead_time' in df.columns:
    high_lead = df[df['lead_time'] > 180]['is_canceled'].mean()
    low_lead = df[df['lead_time'] <= 30]['is_canceled'].mean()
    print(f"   - Lead Time: Long lead times (>180 days) have {high_lead*100:.1f}% cancellation vs {low_lead*100:.1f}% for short lead times")

print("\n3. DATA QUALITY:")
missing_cols = df.isnull().sum()
missing_cols = missing_cols[missing_cols > 0]
if len(missing_cols) > 0:
    print(f"   - Columns with missing values: {len(missing_cols)}")
    print(f"   - Most missing: {missing_cols.idxmax()} ({missing_cols.max()} values)")
else:
    print("   - No missing values found")

print("\n4. FEATURE ENGINEERING OPPORTUNITIES:")
print("   - Create total_nights from weekend + weekday nights")
print("   - Create total_guests from adults + children + babies")
print("   - Extract date features (day of week, season)")
print("   - Create lead_time categories")
print("   - Calculate booking value (ADR × nights)")

print("\n" + "=" * 60)


## Summary

✓ Comprehensive EDA completed
✓ Key patterns and relationships identified
✓ Insights extracted for feature engineering
✓ Data quality assessed

**Next Steps**: Proceed to `03_spark_preprocessing.ipynb` for Spark-based data processing and feature engineering.
